In [1]:
import warnings

warnings.filterwarnings("ignore")

import os
from torch.utils import data
from dataset import*
from config import settings
from torch.utils.tensorboard import SummaryWriter
from model import*
from train_test import *
from sklearn.preprocessing import StandardScaler, MinMaxScaler


## load training data

In [ ]:
transformers = {
        'year': MinMaxScaler(), # Normalize
        'weekday': None,
        'time': None, # Standardlize
        'weather': None
    }
df, transformers = load_data('train', total_sample=None, random_sample=settings.totalN, scaling_transformers=transformers)

loaded csv file shape: (55423856, 8)
setting time info...


In [54]:
df.describe()

,fare_amount,passenger_count,year,weekday,hour,is_holiday,temperature,weathercode,distance,night,...,downtown_pickup_distance,downtown_dropoff_distance,jfk_pickup_distance,jfk_dropoff_distance,ewr_pickup_distance,ewr_dropoff_distance,lgr_pickup_distance,lgr_dropoff_distance,total_fixed_fees,net_fare
count,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,...,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07,5.247511e+07
mean,1.147387e+01,1.692577e+00,4.574819e-01,3.041045e+00,1.348527e+01,3.038481e-01,1.204517e+01,8.867734e+00,2.121438e+00,1.963741e-01,...,7.158603e-02,7.460905e-02,3.003075e-01,3.000146e-01,2.620599e-01,2.636354e-01,1.333274e-01,1.337482e-01,3.384471e+00,8.089403e+00
std,9.510953e+00,1.307598e+00,3.109721e-01,1.948616e+00,6.514131e+00,4.599178e-01,1.047299e+01,1.958844e+01,2.339556e+00,3.972547e-01,...,5.073947e-02,4.973799e-02,4.554721e-02,4.180465e-02,4.914464e-02,5.046139e-02,3.895721e-02,3.961764e-02,7.001533e-01,9.426395e+00
min,2.510000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+01,0.000000e+00,4.442711e-05,0.000000e+00,...,4.000000e-07,5.200000e-06,2.455550e-05,1.244450e-05,8.320000e-04,1.430000e-03,3.700000e-05,6.600000e-05,2.500000e+00,1.000000e-02
25%,6.100000e+00,1.000000e+00,1.666667e-01,1.000000e+00,9.000000e+00,0.000000e+00,3.900000e+00,0.000000e+00,8.195025e-01,0.000000e+00,...,3.868979e-02,4.155560e-02,2.964441e-01,2.955281e-01,2.303570e-01,2.315120e-01,1.135160e-01,1.131050e-01,3.000000e+00,2.700000e+00
50%,8.500000e+00,1.000000e+00,5.000000e-01,3.000000e+00,1.400000e+01,0.000000e+00,1.250000e+01,1.000000e+00,1.377249e+00,0.000000e+00,...,6.521720e-02,6.771520e-02,3.059571e-01,3.058321e-01,2.576212e-01,2.590630e-01,1.304650e-01,1.292370e-01,3.500000e+00,5.100000e+00
75%,1.290000e+01,2.000000e+00,6.666667e-01,5.000000e+00,1.900000e+01,1.000000e+00,2.050000e+01,3.000000e+00,2.481446e+00,0.000000e+00,...,9.183320e-02,9.769820e-02,3.174131e-01,3.175491e-01,2.844400e-01,2.887070e-01,1.564110e-01,1.556340e-01,3.500000e+00,9.500000e+00
max,1.657500e+02,6.000000e+00,1.000000e+00,6.000000e+00,2.300000e+01,1.000000e+00,3.880000e+01,7.500000e+01,7.974493e+01,1.000000e+00,...,2.674394e+00,2.691996e+00,2.520228e+00,2.537830e+00,2.867172e+00,2.884774e+00,2.482172e+00,2.499774e+00,2.130000e+01,1.499500e+02


In [ ]:
print(df.shape)

In [7]:
df.columns

Index(['fare_amount', 'passenger_count', 'year', 'Sunday', 'Monday', 'Tuesday',
       'Wednesday', 'Thursday', 'Friday', 'Saturday', 'weekday', 'hour',
       'is_holiday', 'temperature', 'weathercode', 'distance', 'night',
       'late_night', 'from_JKF', 'to_JKF', 'from_LGA', 'to_LGA', 'to_EWR',
       'from_Manhattan', 'to_Manhattan', 'direction', 'latdiff', 'londiff',
       'euclidean', 'manhattan', 'downtown_pickup_distance',
       'downtown_dropoff_distance', 'jfk_pickup_distance',
       'jfk_dropoff_distance', 'ewr_pickup_distance', 'ewr_dropoff_distance',
       'lgr_pickup_distance', 'lgr_dropoff_distance', 'total_fixed_fees',
       'net_fare'],
      dtype='object')

## Training and validation data loader

In [8]:
BATCH_SIZE = 128
train_dataset = DataFolder(split='train', df=df, transformers=transformers)
train_dataloader = data.DataLoader(dataset=train_dataset,
                                  batch_size=BATCH_SIZE,
                                  shuffle=True,
                                  drop_last=False,
                                  num_workers=4)

In [9]:
valid_dataset = DataFolder(split='valid', df=df, transformers=transformers)
valid_dataloader = data.DataLoader(dataset=valid_dataset,
                                  batch_size=BATCH_SIZE,
                                  shuffle=False,
                                  drop_last=False,
                                  num_workers=0)

## Mode select val or train

In [10]:
Mode_train = True

## Device

In [11]:
device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda:2 device


## Model select

In [12]:
model_name = 'FCNN64_0524_03'
save_model_name = './model/'+ model_name +'.pth'
loss_filename = './loss curve/' + model_name + '.txt'

In [53]:
train_dataset.df.describe()

,fare_amount,passenger_count,year,weekday,hour,is_holiday,temperature,weathercode,distance,night,...,downtown_pickup_distance,downtown_dropoff_distance,jfk_pickup_distance,jfk_dropoff_distance,ewr_pickup_distance,ewr_dropoff_distance,lgr_pickup_distance,lgr_dropoff_distance,total_fixed_fees,net_fare
count,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,...,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07,4.985135e+07
mean,1.147422e+01,1.692666e+00,4.574878e-01,3.041145e+00,1.348495e+01,3.038748e-01,1.204442e+01,8.868145e+00,2.121516e+00,1.963495e-01,...,7.158792e-02,7.460821e-02,3.003051e-01,3.000143e-01,2.620616e-01,2.636339e-01,1.333275e-01,1.337484e-01,3.384476e+00,8.089744e+00
std,9.511529e+00,1.307671e+00,3.109711e-01,1.948608e+00,6.514162e+00,4.599292e-01,1.047314e+01,1.958917e+01,2.339824e+00,3.972359e-01,...,5.074647e-02,4.973034e-02,4.555417e-02,4.179804e-02,4.915215e-02,5.045599e-02,3.896132e-02,3.961169e-02,7.004598e-01,9.426847e+00
min,2.510000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-2.000000e+01,0.000000e+00,4.442711e-05,0.000000e+00,...,4.000000e-07,5.200000e-06,2.455550e-05,1.244450e-05,8.320000e-04,1.430000e-03,3.700000e-05,6.600000e-05,2.500000e+00,1.000000e-02
25%,6.100000e+00,1.000000e+00,1.666667e-01,1.000000e+00,9.000000e+00,0.000000e+00,3.900000e+00,0.000000e+00,8.194663e-01,0.000000e+00,...,3.868820e-02,4.155560e-02,2.964431e-01,2.955273e-01,2.303550e-01,2.315120e-01,1.135150e-01,1.131050e-01,3.000000e+00,2.700000e+00
50%,8.500000e+00,1.000000e+00,5.000000e-01,3.000000e+00,1.400000e+01,0.000000e+00,1.250000e+01,1.000000e+00,1.377266e+00,0.000000e+00,...,6.521720e-02,6.771420e-02,3.059561e-01,3.058321e-01,2.576220e-01,2.590620e-01,1.304660e-01,1.292380e-01,3.500000e+00,5.100000e+00
75%,1.290000e+01,2.000000e+00,6.666667e-01,5.000000e+00,1.900000e+01,1.000000e+00,2.050000e+01,3.000000e+00,2.481454e+00,0.000000e+00,...,9.183520e-02,9.769934e-02,3.174131e-01,3.175481e-01,2.844420e-01,2.887080e-01,1.564130e-01,1.556340e-01,3.500000e+00,9.500000e+00
max,1.657500e+02,6.000000e+00,1.000000e+00,6.000000e+00,2.300000e+01,1.000000e+00,3.880000e+01,7.500000e+01,7.974493e+01,1.000000e+00,...,2.674394e+00,2.691996e+00,2.520228e+00,2.537830e+00,2.867172e+00,2.884774e+00,2.482172e+00,2.499774e+00,2.130000e+01,1.499500e+02


In [14]:
input_dim = np.size(train_dataset.features, 1)
print(f'feature dimension = {input_dim}')
if Mode_train:
    model = FCNN(input_dim=input_dim).to(device) 
else:
    model = FCNN(input_dim=input_dim)
    model.load_state_dict(torch.load(save_model_name))
    model = model.to(device)
print(model)

feature dimension = 38
FCNN(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=38, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=1, bias=True)
  )
)


## Loss function and optimizer

In [15]:
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
# optimizer2 = torch.optim.SGD(model.parameters(), lr=1e-2, weight_decay=0, momentum=0.9)

## Training epoch and stop condition

In [16]:
trained = False

In [17]:
writer = SummaryWriter('./logs/' + model_name)

2023-05-25 00:05:03.729663: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-25 00:05:04.590719: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [18]:
if Mode_train:
    epochs = 1
    if not trained:
        loss_record = [1e6, 1e6, 1e6, 1e6, 1e6]
    for epoch in range(epochs):
        train_loss = train(train_dataloader, model, loss_fn, optimizer, writer=writer, record_batches=200)
        val_loss = val(valid_dataloader, model, loss_fn)
        writer.add_scalar("loss/training", train_loss, epoch+1)
        writer.add_scalar("loss/validation", val_loss, epoch+1)
        print(f"Epoch {epoch + 1:2d}: Loss = {train_loss:.4f}") 
        print(f'valid loss = {val_loss:.4f}')
        loss_record.append(train_loss)
        # if train_loss > sum(loss_record[-5:])/5*1.05 or train_loss < 0.1:
        #     print('Early stop!')
        #     break

    if not trained:        
        del loss_record[0:5]
    trained = True    
    print("Done!")

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20499/20499 [01:18<00:00, 260.00it/s]

Epoch  1: Loss = 3.2855
valid loss = 3.2075
Done!


## Plot training loss curve

In [ ]:
import matplotlib.pyplot as plt

def plot_loss_curve(loss_list):
    plt.plot(loss_list)
    plt.title('Loss Curve')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.show()

In [ ]:
if Mode_train:
    plot_loss_curve(loss_record)
else:
    print('Auto load loss curve')
    with open(loss_filename) as fh:
        s = fh.readline()
        L = s[1:-1].split(', ')
        loss_record = [float(x) for x in L]        
    plot_loss_curve(loss_record)

## Evaluate

In [19]:
#final_train_loss = val(train_dataloader, model, loss_fn)
final_val_loss = val(valid_dataloader, model, loss_fn)
print(f'final valid loss = {final_val_loss:.4f}')

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20499/20499 [01:18<00:00, 260.04it/s]

final valid loss = 3.2075


## Save model

In [20]:
if Mode_train:
    torch.save(model.state_dict(), save_model_name)
    with open(loss_filename, 'w') as fh:
        fh.writelines(str(loss_record))   

## Output test result

In [55]:
test_dataset = DataFolder(split='test', transformers=transformers)
test_dataset.df.replace(-np.inf, -5, inplace=True)
test_dataloader = data.DataLoader(dataset=test_dataset,
                                  batch_size=BATCH_SIZE,
                                  shuffle=False,
                                  drop_last=False,
                                  num_workers=0)

loaded test csv file shape: (9914, 7)
setting time info...
setting geo info...
counting fixed fee...


In [56]:
df_test = pd.DataFrame(test_dataset.key_list, columns=["key"])

In [57]:
test_dataset.df.describe()

,passenger_count,year,weekday,hour,is_holiday,temperature,weathercode,distance,night,late_night,...,manhattan,downtown_pickup_distance,downtown_dropoff_distance,jfk_pickup_distance,jfk_dropoff_distance,ewr_pickup_distance,ewr_dropoff_distance,lgr_pickup_distance,lgr_dropoff_distance,total_fixed_fees
count,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,...,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000,9914.000000
mean,1.671273,0.469303,2.852834,13.467420,0.280916,12.900978,9.147065,2.134640,0.200424,0.400746,...,-3.505500,0.072781,0.076354,0.300563,0.300144,0.263208,0.265217,0.134590,0.133983,3.421192
std,1.278747,0.300558,1.994451,6.868584,0.449469,9.928859,20.238913,2.468319,0.400338,0.490074,...,0.911244,0.065450,0.059750,0.055860,0.048643,0.063896,0.060605,0.051362,0.048436,0.675622
min,1.000000,0.000000,0.000000,0.000000,0.000000,-19.100000,0.000000,0.000000,0.000000,0.000000,...,-10.724468,0.000607,0.000776,0.000151,0.000129,0.161984,0.002922,0.003212,0.000271,2.500000
25%,1.000000,0.166667,1.000000,8.000000,0.000000,5.500000,0.000000,0.806712,0.000000,0.000000,...,-4.074331,0.037943,0.042970,0.296282,0.295326,0.229750,0.232364,0.113397,0.112860,3.000000
50%,1.000000,0.500000,3.000000,15.000000,0.000000,13.900000,1.000000,1.377836,0.000000,0.000000,...,-3.536605,0.064762,0.068687,0.306000,0.305828,0.256905,0.259770,0.131216,0.128524,3.500000
75%,2.000000,0.833333,5.000000,19.000000,1.000000,20.600000,3.000000,2.513634,0.000000,1.000000,...,-2.938185,0.092463,0.100171,0.317276,0.317847,0.284851,0.290071,0.157255,0.155339,3.500000
max,6.000000,1.000000,6.000000,23.000000,1.000000,38.300000,75.000000,62.134660,1.000000,1.000000,...,0.392018,2.015245,1.997942,1.861079,1.843776,2.208023,2.190720,1.823023,1.805720,21.000000


In [58]:
predictions = test(test_dataloader, model)

df_test["fare_amount"] = predictions
# df_test = df_test.fillna(0)
df_test["fare_amount"] += test_dataset.df["total_fixed_fees"]
df_test.loc[df_test['fare_amount'] < 2.8, 'fare_amount'] = 2.8
print(df_test.isnull().sum().sum())
#df_test = df_test.fillna(10)
# 將dataframe保存為CSV文件
df_test.to_csv("predictions.csv", index=False)
print('output complete!')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 265.79it/s]


85
output complete!


In [52]:
df_test.describe()

,fare_amount
count,9829.000000
mean,11.373303
std,8.742736
min,3.566613
25%,6.366226
50%,8.533180
75%,12.549000
max,110.725224


In [50]:
print(test_dataset.df.values[279])

['2010-09-05 22:31:32.0000002' 1 0.16666666666668561 False False False
 False False True False 6 22 1 24.3 0.0 0.0 0 1 False False False False
 False False False 0.0 0.0 0.0 -10.0 -10.0 0.11262739999999383
 0.11262739999999383 0.4067941110999911 0.4067941110999911
 0.22339500000000356 0.22339500000000356 0.19318299999999056
 0.19318299999999056 3.5]
